In [1]:
from __future__ import division, unicode_literals, print_function
import numpy as np
import pandas as pd

# 2. 心理学実験のための知識
基本的に，実験は，

* 出力データの作成
* 呈示刺激の準備
* 試行回数ぶんのforループ

で構成されます．

## 2.1. 出力データの作成
### 2.1.1. 最初のデータフレーム

In [2]:
index = pd.MultiIndex.from_product([['a1', 'a2'], ['b1', 'b2', 'b3']], names=['A', 'B'])
columns = ['subj', 'RT']
df = pd.DataFrame(0, index=index, columns=columns)
df

subj  RT
A  B           
a1 b1     0   0
   b2     0   0
   b3     0   0
a2 b1     0   0
   b2     0   0
   b3     0   0

### 2.1.2. 被験者番号をつける

In [3]:
df['subj'] = 1
df

subj  RT
A  B           
a1 b1     1   0
   b2     1   0
   b3     1   0
a2 b1     1   0
   b2     1   0
   b3     1   0

### 2.1.3. 複製

In [4]:
pd.concat([df, df])  # dfとdfが縦に連結 (concatenate) された

subj  RT
A  B           
a1 b1     1   0
   b2     1   0
   b3     1   0
a2 b1     1   0
   b2     1   0
   b3     1   0
a1 b1     1   0
   b2     1   0
   b3     1   0
a2 b1     1   0
   b2     1   0
   b3     1   0

In [5]:
li = []
for i in range(3):
    li.append(df)  # 4コのデータフレームからなるlistを作成して，
pd.concat(li)  # それらを全て結合し「繰り返し4回」を実現する

subj  RT
A  B           
a1 b1     1   0
   b2     1   0
   b3     1   0
a2 b1     1   0
   b2     1   0
   b3     1   0
a1 b1     1   0
   b2     1   0
   b3     1   0
a2 b1     1   0
   b2     1   0
   b3     1   0
a1 b1     1   0
   b2     1   0
   b3     1   0
a2 b1     1   0
   b2     1   0
   b3     1   0

### 2.1.4. ランダマイズ

In [6]:
n_trial = len(df)
randomized_index = np.random.permutation(n_trial)  # ランダムな並び替えを取得
randomized_index

array([3, 2, 5, 1, 4, 0])

In [7]:
df = df.iloc[randomized_index, :]
df

,,subj,RT
A,B,,
a2,b1,1,0
a1,b3,1,0
a2,b3,1,0
a1,b2,1,0
a2,b2,1,0
a1,b1,1,0


## 2.2. 例: Stroop課題の実験用データフレームを作成する

In [8]:
SUBJCT_ID = 1
N_REPEAT = 3

index = pd.MultiIndex.from_product([['R', 'G', 'B'], ['赤', '緑', '青']], names=['ink', 'text'])
columns = ['subj', 'RT']
df = pd.DataFrame(0, index=index, columns=columns)

# 繰り返し 
li = []
for i in range(N_REPEAT):
    li.append(df)
df = pd.concat(li)  # or df = pd.concat([df for i in range(N_REPEAT)])

# 被験者IDをつけておく
df['subj'] = SUBJCT_ID

# ランダマイズ
randomized_index = np.random.permutation(len(df))
df = df.iloc[randomized_index, :]

In [9]:
df  # 3 * 3水準，繰り返し3回 = 27行のデータフレームが作成された．

subj  RT
ink text          
G   青        1   0
B   緑        1   0
R   緑        1   0
G   青        1   0
R   緑        1   0
G   赤        1   0
    赤        1   0
B   青        1   0
    赤        1   0
    赤        1   0
    緑        1   0
R   赤        1   0
B   青        1   0
R   青        1   0
    赤        1   0
    青        1   0
G   緑        1   0
    緑        1   0
R   緑        1   0
G   緑        1   0
    青        1   0
B   青        1   0
    緑        1   0
G   赤        1   0
R   青        1   0
    赤        1   0
B   赤        1   0

## 2.2. 呈示刺激の準備
(省略)

## 2.3. 試行回数ぶんのforループ

### 2.3.1. df.iterrows()
df.iterrows()は，データフレームを1行目から順番に返す．

In [10]:
for cond, value in df[:3].iterrows():  # dfの頭3行
    print('================')
    print(cond)
    print('----------------')
    print(value)

('G', '青')
----------------
subj    1
RT      0
Name: (G, 青), dtype: int64
('B', '緑')
----------------
subj    1
RT      0
Name: (B, 緑), dtype: int64
('R', '緑')
----------------
subj    1
RT      0
Name: (R, 緑), dtype: int64


In [11]:
value.subj

1

In [12]:
value.RT

0

### 2.3.2. 値の代入
df.iterrows()は"参照渡し"を行うため，condやvalueを変更するともとのdfも変更される．

In [13]:
value.RT = 777

In [14]:
df.head()  # 頭5行を表示

,,subj,RT
ink,text,,
G,青,1,0
B,緑,1,0
R,緑,1,777
G,青,1,0
R,緑,1,0


これを利用し，実験ループごとに反応時間を取得，代入していく